# Lesson 4: Question answering

![](./images/rag_diagram.png)

In [14]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [15]:
import { loadAndSplitChunks } from "./lib/helpers.ts";

const splitDocs = await loadAndSplitChunks({
    chunkSize: 1536,
    chunkOverlap: 128
});

In [16]:
import { initializeVectorstoreWithDocuments } from "./lib/helpers.ts";

const vectorstore = await initializeVectorstoreWithDocuments({
  documents: splitDocs,
});

In [17]:
const retriever = vectorstore.asRetriever();

# Document retrieval in a chain

In [18]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";

const convertDocsToString = (documents: Document[]): string => {
  return documents.map((document) => {
    return `<doc>\n${document.pageContent}\n</doc>`
  }).join("\n");
};

/*
{
question: "What is deep learning?"
}
*/

const documentRetrievalChain = RunnableSequence.from([
    (input) => input.question,
    retriever,
    convertDocsToString
]);

In [19]:
const results = await documentRetrievalChain.invoke({
  question: "What are the prerequisites for this course?"
});
console.log(results);

<doc>
course information handout. So let me just say a few words about parts of these. On the 
third page, there's a section that says Online Resources.  
Oh, okay. Louder? Actually, could you turn up the volume? Testing. Is this better? 
Testing, testing. Okay, cool. Thanks.
</doc>
<doc>
of this class will not be very programming intensive, although we will do some 
programming, mostly in either MATLAB or Octave. I'll say a bit more about that later.  
I also assume familiarity with basic probability and statistics. So most undergraduate 
statistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna 
assume all of you know what random variables are, that all of you know what expectation 
is, what a variance or a random variable is. And in case of some of you, it's been a while 
since you've seen some of this material. At some of the discussion sections, we'll actually 
go over some of the prerequisites, sort of as a refresher course under prerequisite cl

# Synthesizing a response

In [20]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const TEMPLATE_STRING = `You are an experienced researcher, 
expert at interpreting and answering questions based on provided sources.
Using the provided context, answer the user's question 
to the best of your ability using only the resources provided. 
Be verbose!

<context>

{context}

</context>

Now, answer this question using the above context:

{question}`;

const answerGenerationPrompt = ChatPromptTemplate.fromTemplate(
    TEMPLATE_STRING
);

In [21]:
import { RunnableMap } from "@langchain/core/runnables";

const runnableMap = RunnableMap.from({
  context: documentRetrievalChain,
  question: (input) => input.question,
});

await runnableMap.invoke({
    question: "What are the prerequisites for this course?"
})

{
  question: "What are the prerequisites for this course?",
  context: "<doc>\n" +
    "course information handout. So let me just say a few words about parts of these. On the \n" +
    "third"... 3063 more characters
}

# Augmented generation

In [22]:
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";

const model = new ChatOpenAI({
    modelName: "gpt-3.5-turbo-1106"
});

In [23]:
const retrievalChain = RunnableSequence.from([
  {
    context: documentRetrievalChain,
    question: (input) => input.question,
  },
  answerGenerationPrompt,
  model,
  new StringOutputParser(),
]);

In [24]:
const answer = await retrievalChain.invoke({
  question: "What are the prerequisites for this course?"
});

console.log(answer);

The prerequisites for this course include familiarity with basic probability and statistics, as well as basic linear algebra. In terms of statistics, it is assumed that students are familiar with concepts such as random variables, expectation, variance, and other related material. Familiarity with linear algebra is also expected, with students being required to understand topics such as matrices, vectors, multiplying matrices and vectors, matrix inverses, and eigenvectors of a matrix. It is also mentioned that undergraduate statistics classes like Stat 116 at Stanford and undergraduate linear algebra courses like Math 51, 103, Math 113, or CS205 at Stanford would be more than enough to meet the prerequisites for the course. Additionally, the instructor mentions that review sections will be held for some of the prerequisites to serve as a refresher course for students who may not be as familiar with the material.


In [25]:
const followupAnswer = await retrievalChain.invoke({
  question: "Can you list them in bullet point form?"
});

console.log(followupAnswer);

Based on the provided context, it is not clear what specific items or topics the user is asking to be listed in bullet point form. The context includes references to course information, supervised learning, machine learning projects, and using different functions to predict house prices. If the user can provide more specific details about what they would like to be listed in bullet point form, I would be happy to assist further.


In [ ]:
const docs = await documentRetrievalChain.invoke({
  question: "Can you list them in bullet point form?"
});

console.log(docs);